# Processamento de Linguagem Natural - Faculdade Descomplica
# Aula 13 - Pipelines de Linguagem Natural e Processamento em Lote

Prof. Elvis de Souza

# Carregando bibliotecas

In [ ]:
! pip3 install spacy
! python3 -m spacy download pt_core_news_sm
! pip3 install wikipedia

In [ ]:
import spacy
import wikipedia
import json

wikipedia.set_lang("pt")
nlp = spacy.load("pt_core_news_sm")

# Carregando texto

In [ ]:
texto = wikipedia.page("PLN")

In [ ]:
texto

In [ ]:
texto.content

# Processando texto com spaCy e explorando funções

In [ ]:
doc = nlp(texto.content)

In [ ]:
type(doc)

In [ ]:
doc[0]

In [ ]:
len(doc)

In [ ]:
print(doc.to_json())

In [ ]:
print(json.dumps(doc.to_json(), indent=4, ensure_ascii=False))

In [ ]:
doc[0].pos_

In [ ]:
doc[0].dep_

In [ ]:
doc[0].head

In [ ]:
(doc[0].text, doc[0].pos_ + "/" + doc[0].dep_, doc[0].head)

# Criando funções que irão compor o pipeline

In [ ]:
def extrair_sintaxe(doc):
    saida = []
    for token in doc:
        saida.append((token.text, token.pos_ + "/" + token.dep_, token.head.text))
    return saida

In [ ]:
extrair_sintaxe(doc)

In [ ]:
def extrair_sujeitos(doc):
    saida = []
    for token in doc:
        if token.dep_ == "nsubj":
            saida.append((token.text, token.head.text))
    return saida

In [ ]:
extrair_sujeitos(doc)

# Definição do pipeline

In [ ]:
def pipeline(texto, nlp):
    doc = nlp(texto)
    saida = doc.to_json()
    saida['sintaxe'] = extrair_sintaxe(doc)
    saida['sujeitos'] = extrair_sujeitos(doc)
    return saida

In [ ]:
pipeline("Processamento de Linguagem Natural é uma subárea da inteligência artificial.", nlp)

# Vídeo 4: Desenvolvimento de um pipeline completo de PLN em Python

In [ ]:
import os
import uuid
import datetime

# Criando pastas e arquivo de log

In [ ]:
if not os.path.isdir("raw_texts"):
    os.mkdir("raw_texts")

if not os.path.isdir("processed_texts"):
    os.mkdir("processed_texts")

log_file = open("log.txt", "a")

# Extraindo documentos

In [ ]:
paginas = []
for letra in "abcdefghijklmnopqrstuvwxyz":
    paginas.extend(wikipedia.search(letra, results=5))

In [ ]:
len(paginas)

In [ ]:
paginas

In [ ]:
uuid.uuid4()

In [ ]:
import tqdm
# ATENÇÃO: Demora 2 minutos.
for pagina in tqdm.tqdm(paginas):
    try:
        texto = wikipedia.page(pagina).content
    except wikipedia.DisambiguationError:
        log_file.write(f"[{str(datetime.datetime.now())}] DisambiguationError: {pagina}\n")
    except wikipedia.PageError:
        log_file.write(f"[{str(datetime.datetime.now())}] PageError: {pagina}\n")
    else:
        with open(f"raw_texts/{uuid.uuid4()}.txt", "w") as f:
            f.write(texto)
        log_file.write(f"[{str(datetime.datetime.now())}] OK: {pagina}\n")

In [ ]:
len(os.listdir("raw_texts"))

# Transformando documentos e armazenando

In [ ]:
import json

nlp = spacy.load("pt_core_news_sm")

for arquivo in tqdm.tqdm(os.listdir("raw_texts")):
    with open("raw_texts/" + arquivo) as f:
        texto = f.read()

    processado = pipeline(texto, nlp)

    timestamp = str(datetime.datetime.now())
    processado['timestamp'] = timestamp
    log_file.write(f"[{str(datetime.datetime.now())}] Pipeline: {arquivo}\n")

    with open("processed_texts/" + arquivo + ".json", "w") as f:
        json.dump(processado, f)

    os.remove("raw_texts/" + arquivo)

In [ ]:
log_file.close()